# Creating a real Agent
* Unlike the previous one, this Agent does have agentic behavior.
* This is the code the LangChain Team provides in the LangChain Documentation.
* Do not get overwhelmed by the following code. We will explain it briefly below, and in the following lessons we will study each of this components and more in detail.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy


# Define system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# Define tools
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

# Configure model
model = init_chat_model(
    "gpt-4o-mini",
    temperature=0.0
)

# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

# Set up memory
checkpointer = InMemorySaver()

# Create agent
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

# See what's actually in the response
print("Response keys:", response.keys())
print("Last message:", response['messages'][-1])

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

Response keys: dict_keys(['messages', 'structured_response'])
Last message: content='Returning structured response: ResponseFormat(punny_response="Looks like Florida is having a bright day! It\'s always sunny in the Sunshine State!", weather_conditions=\'sunny\')' name='ResponseFormat' id='0a44462a-d80f-4d9b-8fc8-b3d9939db61b' tool_call_id='call_rQT4FINdG7AwupUxSXwGoDXT'
ResponseFormat(punny_response="Looks like Florida is having a bright day! It's always sunny in the Sunshine State!", weather_conditions='sunny')
ResponseFormat(punny_response="You're welcome! I'm always here to help you weather the storm!", weather_conditions=None)


## Let's explain in simple terms the previous code included in the LangChain 1.0 Quickstart Guide
* Do not get overwhelmed, we use this just as a quick demo of a real agent. We will explain the components of an Agent in detail in the following lessons. 

#### Step 1: Import Required Modules

```python
from dataclasses import dataclass
```
**What it does:** Imports Python's `dataclass` decorator, which makes it easy to create simple classes that just store data.


```python
from langchain.agents import create_agent
```
**What it does:** Imports the main function to create your AI agent.


```python
from langchain.chat_models import init_chat_model
```
**What it does:** Imports the function to initialize your AI model (Claude in this case).


```python
from langchain.tools import tool, ToolRuntime
```
**What it does:** 
- `tool` - A decorator that turns regular Python functions into tools the AI can use
- `ToolRuntime` - Allows tools to access runtime information (like user context)


```python
from langgraph.checkpoint.memory import InMemorySaver
```
**What it does:** Imports the memory system so your agent can remember previous conversations.


```python
from langchain.agents.structured_output import ToolStrategy
```
**What it does:** Imports a helper to make sure the agent returns responses in a specific format you define.


#### Step 2: Define the System Prompt

```python
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""
```

**What it does:** This is the "personality" and instructions for your AI agent.
- Tells the AI to act like a weather forecaster
- Tells it to use puns (for fun!)
- Gives instructions on when to use the location tool

**Think of it as:** The job description for your AI employee.


#### Step 3: Define the Context Schema

```python
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str
```

**What it does:** Creates a simple data container that holds user information.
- `user_id` is a string that identifies which user is talking to the agent
- This allows the agent to personalize responses based on who's asking

**Example:** If `user_id = "1"`, the agent knows this is user #1.


#### Step 4: Define Tools (Functions the AI Can Use)

```python
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"
```

**Line-by-line:**
- `@tool` - Marks this function as a tool the AI can call
- `def get_weather_for_location(city: str) -> str:` - Defines a toy function that takes a city name and returns text
- `"""Get weather for a given city."""` - Documentation the AI reads to understand what this tool does
- `return f"It's always sunny in {city}!"` - **This is just a toy function that returns a fake weather report (in a real app, this would call a weather API)**

---

```python
@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"
```

**Line-by-line:**
- `@tool` - Marks this as another tool
- `runtime: ToolRuntime[Context]` - This special parameter gives the function access to the user's context
- `user_id = runtime.context.user_id` - Extracts the user ID from the context
- `return "Florida" if user_id == "1" else "SF"` - Returns location based on user:
  - If user ID is "1", return "Florida"
  - Otherwise, return "SF" (San Francisco)
  - **This is just a toy function. In a real app, this would be a function to get the user location**.

**Why this matters:** The AI can figure out where the user is without them saying it!


#### Step 5: Configure the AI Model

```python
model = init_chat_model(
    "gpt-4o-mini",
    temperature=0.0
)
```

**Line-by-line:**
- `model =` - Creates a variable to store your AI model
- `init_chat_model()` - Initializes the chat model
- `"gpt-4o-mini"` - Specifies which model to use (gpt-4o-mini)
- `temperature=0.0` - Controls randomness:
  - `0.0` = very consistent, predictable responses


#### Step 6: Define the Response Format (the Structured Output)

```python
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None
```

**What it does:** Defines exactly how the agent should structure its responses.

**Line-by-line:**
- `punny_response: str` - A text field for the punny response (required)
- `weather_conditions: str | None = None` - An optional field for weather info
  - `str | None` means it can be text or nothing
  - `= None` makes it optional

**Example output:**
```python
ResponseFormat(
    punny_response="Florida is having a sun-derful day!",
    weather_conditions="It's always sunny in Florida!"
)
```


#### Step 7: Set Up Short-Term Memory (Conversation Memory)

```python
checkpointer = InMemorySaver()
```

**What it does:** Creates a memory system that saves conversation history.
- Allows the agent to remember what was said earlier
- Stored in memory (RAM) - resets when your program stops

**Why it matters:** Without this, the agent would forget everything between messages!


#### Step 8: Create the Agent

```python
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)
```

This is where everything comes together.

**Line-by-line:**
- `model=model` - Uses the Claude model you configured
- `system_prompt=SYSTEM_PROMPT` - Gives the agent its instructions and personality
- `tools=[...]` - Provides the list of tools the agent can use
- `context_schema=Context` - Tells the agent what user information it has access to
- `response_format=ToolStrategy(ResponseFormat)` - Ensures responses follow your defined structure
- `checkpointer=checkpointer` - Enables conversation memory


#### Step 9: Run the Agent with Conversation ID (for Short-Term Memory) and Custom Context (in this case, some user information to remember)

```python
config = {"configurable": {"thread_id": "1"}}
```
**What it does:** Creates a configuration that identifies this conversation thread.
- `thread_id` acts like a "conversation ID"
- All messages with the same thread_id are part of the same conversation

---

```python
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)
```

**Line-by-line:**
- `agent.invoke()` - Sends a message to the agent and gets a response
- `{"messages": [...]}` - The conversation format:
  - `"role": "user"` - This message is from the user
  - `"content": "..."` - The actual question
- `config=config` - Uses the thread_id we defined
- `context=Context(user_id="1")` - Tells the agent this is user #1


**What happens behind the scenes:**
1. Agent reads the question
2. Realizes it needs to know the user's location
3. Calls `get_user_location()` tool (returns "Florida" for user 1)
4. Calls `get_weather_for_location("Florida")` tool
5. Generates a punny response with the weather info

---

```python
print(response['structured_response'])
```
**What it does:** Prints the structured response in the format you defined.

**Example output:**
```python
ResponseFormat(
    punny_response="Florida is having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long!",
    weather_conditions="It's always sunny in Florida!"
)
```


#### Key Concepts Summary

1. **Tools** = Functions the AI can call when needed
2. **System Prompt** = Instructions that guide the AI's behavior
3. **Context** = User-specific information the AI can access
4. **Response Format** = The structure of the AI's answers
5. **Memory** = Allows the AI to remember previous messages
6. **Thread ID** = Identifies a specific conversation


#### Try It Yourself

You can continue the conversation:

```python
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Why is it almost always sunny in Florida?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
```

The agent will remember the previous conversation and respond accordingly.

## How to run this code from Visual Studio Code
* Open Terminal.
* Make sure you are in the project folder.
* Make sure you have the poetry env activated.
* Enter and run the following command:
    * `python 003-creating-a-real-agent.py` 